In [2]:
import torch.nn as nn 
from torch.utils.data import Dataset
from torch.utils.data import dataloader

/home/wzm289/miniconda3/envs/ACL/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# load dataset
import pandas as pd 
df = pd.read_csv("../scifact/train.source_rl", sep = "\t", names = ["source"])

In [5]:
# test the gpt tokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
prep_txt = "hello world"
modelname = "gpt2"

# load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(modelname, bos_token = "<|startoftext|>",
eos_token = "<|endoftext|>",pad_token='<|pad|>')

encode_tokenizer = tokenizer(df["source"][:10].to_list(), truncation=True,max_length=20, padding="max_length",return_tensors="pt")
encode_tokenizer['input_ids']

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/tmp/ipykernel_815284/523614133.py:10: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  encode_tokenizer = tokenizer(df["source"][:10].to_list(), truncation=True,max_length=20, padding="max_length",return_tensors="pt")


tensor([[ 6030,   352, 39569,   318,  3917,   351, 11800, 22146,  5945,   602,
           287,   309,   842,  2478,    13, 50258, 50258, 50258, 50258, 50258],
        [  818,  1007, 38516, 10693, 25451,   278,  4077,   781, 35414,  7532,
           739,   262,  1630,   286,   262, 17689,    17, 32972,    11,   517],
        [38208,   312,   876,  7446,  2465, 36206,  3563,  2751, 45829,    13,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258],
        [ 1404,    44,   290,  5325,    18,  3519,  7532,   389,  4688,   329,
         34244,  7446,  2465,    13, 50258, 50258, 50258, 50258, 50258, 50258],
        [ 8021, 27289,  3513, 34721,   318,   517, 13205,   284,  8668,  3357,
           621, 15964,  8027, 10906,    13, 50258, 50258, 50258, 50258, 50258],
        [   45, 28978,    40, 23005,  2728,  6625,   284,   497, 37040,   499,
           500,    13, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258],
        [28780,   934, 29906,   389,   407, 33

In [6]:
# gpt mode load
model = GPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=tokenizer.eos_token_id)
encode_tokenizer['input_ids']

tensor([[ 6030,   352, 39569,   318,  3917,   351, 11800, 22146,  5945,   602,
           287,   309,   842,  2478,    13, 50258, 50258, 50258, 50258, 50258],
        [  818,  1007, 38516, 10693, 25451,   278,  4077,   781, 35414,  7532,
           739,   262,  1630,   286,   262, 17689,    17, 32972,    11,   517],
        [38208,   312,   876,  7446,  2465, 36206,  3563,  2751, 45829,    13,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258],
        [ 1404,    44,   290,  5325,    18,  3519,  7532,   389,  4688,   329,
         34244,  7446,  2465,    13, 50258, 50258, 50258, 50258, 50258, 50258],
        [ 8021, 27289,  3513, 34721,   318,   517, 13205,   284,  8668,  3357,
           621, 15964,  8027, 10906,    13, 50258, 50258, 50258, 50258, 50258],
        [   45, 28978,    40, 23005,  2728,  6625,   284,   497, 37040,   499,
           500,    13, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258],
        [28780,   934, 29906,   389,   407, 33

In [10]:
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token # to avoid an error

sentences = ["Hello, my dog is a little",
            "Hello, my dog is", # use different length sentences to test batching
            ]
inputs = tokenizer(sentences, return_tensors="pt", padding=True)


output_sequences = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    do_sample=False, # disable sampling to test if batching affects output
)

for i in range(len(sentences)):
    print(tokenizer.decode(output_sequences[i]), "**********")

/home/wzm289/miniconda3/envs/ACL/lib/python3.8/site-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Hello, my dog is a little bit of a mess. I'm not sure if he's going **********
<|endoftext|><|endoftext|>Hello, my dog is a little bit of a mess. I'm not sure if he **********


In [8]:

model.generate(input_ids = encode_tokenizer["input_ids"],
attention_mask = encode_tokenizer["attention_mask"],
do_sample = False,
pad_token_id=tokenizer.eos_token_id)

/home/wzm289/miniconda3/envs/ACL/lib/python3.8/site-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 20, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


IndexError: index out of range in self

In [7]:
df_source = pd.read_csv("../scifact/train.source",sep = "\t",names = ["source"])
df_target = pd.read_csv("../scifact/train.target",sep = "\t",names = ["target"])
df_train = "<|startoftext|>Source:" + df_source["source"] + "<|pad|>Target:" + df_target["target"] + "<|endoftext|>"
for item in df_train[:1]:
    print(item)

<|startoftext|>Source:Type 1 Diabetes is associated with subtle perturbations in T reg development.<|pad|>Target:Autoimmune diseases are thought to result from imbalances in normal immune physiology and regulation. Here, we show that autoimmune disease susceptibility and resistance alleles on mouse chromosome 3 (Idd3) correlate with differential expression of the key immunoregulatory cytokine interleukin-2 (IL-2). In order to test directly that an approximately twofold reduction in IL-2 underpins the Idd3-linked destabilization of immune homeostasis, we show that engineered haplodeficiency of Il2 gene expression not only reduces T cell IL-2 production by twofold but also mimics the autoimmune dysregulatory effects of the naturally occurring susceptibility alleles of Il2. Reduced IL-2 production achieved by either genetic mechanism correlates with reduced function of CD4+ CD25+ regulatory T cells, which are critical for maintaining immune homeostasis.<|endoftext|>


In [8]:
len(df_source)

735

In [9]:
len(df_target)

735

In [30]:
# get the dataset
import pandas as pd 
import torch
class S2Sdataset(Dataset):
    def __init__(self,max_length = 20, data_type = "train"):
        self.input_ids = []
        self.attn_masks = []

        df_source = pd.read_csv("scifact/train.source",sep = "\t",names = ["source"])
        df_target = pd.read_csv("scifact/train.target",sep = "\t",names = ["target"])
        df_train = "<|startoftext|>Source:" + df_source["source"] + "<|pad|>Target:" + df_target["target"] + "<|endoftext|>"
    

        for item in df_train:
            # tokenize
            encodings_dict = tokenizer(item, truncation=True,
                                       max_length = max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, index):
        return self.input_ids[index], self.attn_masks[index]
        



In [31]:
train_dataset = S2Sdataset(data_type="train")
test_dataset = S2Sdataset(data_type = "test")

In [32]:
for test in test_dataset:
    print(test)

(tensor([50257,  7416,    25,  6030,   352, 39569,   318,  3917,   351, 11800,
        22146,  5945,   602,   287,   309,   842,  2478,    13, 50258, 21745]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))
(tensor([50257,  7416,    25,   818,  1007, 38516, 10693, 25451,   278,  4077,
          781, 35414,  7532,   739,   262,  1630,   286,   262, 17689,    17]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))
(tensor([50257,  7416,    25, 38208,   312,   876,  7446,  2465, 36206,  3563,
         2751, 45829,    13, 50258, 21745,    25,   464,  2597,   286,   379]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))
(tensor([50257,  7416,    25,  1404,    44,   290,  5325,    18,  3519,  7532,
          389,  4688,   329, 34244,  7446,  2465,    13, 50258, 21745,    25]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))
(tensor([50257,  7416,    25,  8021, 27289,  3513, 34721,   318,   517, 13205,
     

# training the model

In [37]:
# creating the train arguments
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir='results', num_train_epochs=10, logging_steps=10,
                                 load_best_model_at_end=True, save_strategy="epoch", evaluation_strategy="epoch",
                                 per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                 warmup_steps=100, weight_decay=0.01, logging_dir='logs')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [38]:
model = GPT2LMHeadModel.from_pretrained("gpt2").cuda()
model.resize_token_embeddings(len(tokenizer))

loading configuration file config.json from cache at /home/wzm289/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.25.1",
  "u

Embedding(50259, 768)

In [39]:
# start training
Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset,
        data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                    'attention_mask': torch.stack([f[1] for f in data]),
                                    'labels': torch.stack([f[0] for f in data])}).train()

/home/wzm289/miniconda3/envs/ACL/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 735
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1840
  Number of trainable parameters = 124441344
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/home/wzm289/miniconda3/envs/ACL/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimensi

Epoch,Training Loss,Validation Loss
1,3.532300,2.840172
2,2.700800,2.185281
3,2.447100,1.813298
4,1.804300,1.463862
5,1.775600,1.228431
6,1.460600,1.025475
7,1.287900,0.876116
8,1.204000,0.770225
9,1.082800,0.714552
10,1.024600,0.693485


***** Running Evaluation *****
  Num examples = 735
  Batch size = 4
Saving model checkpoint to results/checkpoint-184
Configuration saved in results/checkpoint-184/config.json
Model weights saved in results/checkpoint-184/pytorch_model.bin
/home/wzm289/miniconda3/envs/ACL/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 735
  Batch size = 4
Saving model checkpoint to results/checkpoint-368
Configuration saved in results/checkpoint-368/config.json
Model weights saved in results/checkpoint-368/pytorch_model.bin
/home/wzm289/miniconda3/envs/ACL/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a ve

TrainOutput(global_step=1840, training_loss=2.722032918619073, metrics={'train_runtime': 995.0431, 'train_samples_per_second': 7.387, 'train_steps_per_second': 1.849, 'total_flos': 75019392000000.0, 'train_loss': 2.722032918619073, 'epoch': 10.0})

In [46]:
# Test 

model.eval()

text = "A deficiency of vitamin B12 increases blood levels of homocysteine."
prompt = f'<|startoftext|>Source: {text}\nTarget:'

generated = tokenizer(f"{prompt}", return_tensors="pt").input_ids.cuda()

# perform prediction
sample_outputs = model.generate(generated, do_sample=False, top_k=50, max_length=50, top_p=0.90, 
        temperature=0, num_return_sequences=0)
# decode the predicted tokens into texts
predicted_text  = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [47]:
predicted_text

'Source: A deficiency of vitamin B12 increases blood levels of homocysteine.\nTarget: To explore the role of vitamin BTarget: The present study investigated the effect of a highA deficiencyTarget: The present data indicated'